<img src="http://hilpisch.com/tpq_logo.png" alt="The Python Quants" width="35%" align="right" border="0"><br>

# Python for Quantitative Finance

&copy; Dr. Yves J. Hilpisch | The Python Quants GmbH

http://tpq.io | [training@tpq.io](mailto:trainin@tpq.io) | [@dyjh](http://twitter.com/dyjh)

# Machine Learning

With **`scikit-learn`**.

In [ ]:
import numpy as np
import pandas as pd
from pylab import plt
plt.style.use('seaborn')
%config InlineBackend.figure_format = 'svg'

## Regime Detection

Based on **unsupervised learning** (= features only).

Regimes could be, for example:

* low volatility, positive trend
* high volatility, positive trend
* low volatility, negative trend
* high volatility, negative trend

### The Data

In [ ]:
url = 'https://certificate.tpq.io/mlfin.csv'

In [ ]:
raw = pd.read_csv(url, index_col=0, parse_dates=True)

In [ ]:
data = pd.DataFrame(raw['.SPX']).dropna()

In [ ]:
data.info()

In [ ]:
data.plot();

In [ ]:
data['r'] = np.log(data / data.shift(1))

### The Features

In [ ]:
window = 40

In [ ]:
data['m'] = data['r'].rolling(window).mean()  # rolling momentum/trend

In [ ]:
data['v'] = data['r'].rolling(window).std()  # rolling volatility

In [ ]:
data.head()

In [ ]:
data.dropna(inplace=True)

In [ ]:
data[['m', 'v']].plot();

In [ ]:
data['m'] = data['m'] * 252
data['v'] = data['v'] * 252 ** 0.5

In [ ]:
f = ['m', 'v']

In [ ]:
data[f] = (data[f] - data[f].mean()) / data[f].std()  # normalization

In [ ]:
data[f].plot.scatter(x='v', y='m');

## Clustering

In [ ]:
from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture

In [ ]:
model = KMeans(n_clusters=4)

In [ ]:
model.fit(data[f])

In [ ]:
model.predict(data[f])

In [ ]:
data['p'] = model.predict(data[f])

In [ ]:
data[f].plot.scatter(x='v', y='m', c=data['p'], cmap='coolwarm');

In [ ]:
plt.scatter(x=data.index, y=data['.SPX'], c=data['p'],
            marker='.', cmap='coolwarm');

## Black-Scholes-Merton

Based on **supervised learning** (features and labels data). 

The idea is to learn the BSM (1973) option prices from parameter and value data.

In [ ]:
from itertools import product

In [ ]:
# on Google Colab execute:
#!wget https://raw.githubusercontent.com/yhilpisch/py4qf/main/bsm.py

In [ ]:
from bsm import bsm_call_value

In [ ]:
bsm_call_value(S0=100, K=105, T=1, r=0.05, sigma=0.2)

In [ ]:
S0_ = np.linspace(80, 120, 3)
S0_

In [ ]:
K_ = np.linspace(80, 120, 3)
K_

In [ ]:
T_ = np.linspace(0.5, 1.5, 3)
T_

In [ ]:
r_ = np.linspace(0.0, 0.05, 3)
r_

In [ ]:
sigma_ = np.linspace(0.1, 0.2, 3)
sigma_

In [ ]:
data = pd.DataFrame()

In [ ]:
%%time
for S0, K, T, r, sigma in product(S0_, K_, T_, r_, sigma_):
    value = bsm_call_value(S0, K, T, r, sigma)
    res = pd.DataFrame({'S0': S0, 'K': K, 'T': T, 'r': r,
                        'sigma': sigma, 'value': value}, index=[0])
    data = pd.concat((data, res), ignore_index=True)

In [ ]:
data.info()

In [ ]:
data.head()

In [ ]:
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error

In [ ]:
model = MLPRegressor(hidden_layer_sizes=[512, 512, 512],
                     max_iter=2000, learning_rate_init=0.0001,
                     activation='tanh')

In [ ]:
f = list(data.columns[:5])
f

In [ ]:
data_ = (data - data.mean()) / data.std()

In [ ]:
%time model.fit(data_[f], data['value'])

In [ ]:
data['estimate'] = model.predict(data_[f])

In [ ]:
data.head()

In [ ]:
mean_squared_error(data['value'], data['estimate'])

In [ ]:
data[['value', 'estimate']].plot();

<img src="http://hilpisch.com/tpq_logo.png" alt="The Python Quants" width="35%" align="right" border="0"><br>

<a href="http://tpq.io" target="_blank">http://tpq.io</a> | <a href="http://twitter.com/dyjh" target="_blank">@dyjh</a> | <a href="mailto:training@tpq.io">training@tpq.io</a>